In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#기본
import pandas as pd
import os
import numpy as np
import math

In [ ]:
# gis
import geopandas as gpd
from shapely import wkt
import osmnx as ox
import networkx as nx
import json

In [ ]:
#시각화
import plotly.express as px
import plotly.io as po
from plotly import graph_objs as go

In [ ]:
import statsmodels.api as sm

### 데이터로드

In [ ]:
complaints_df=pd.read_csv('../1.데이터/2.usedata/민원데이터.csv',engine='python')
school_df=pd.read_csv('../1.데이터/2.usedata/학교데이터.csv',engine='python')
parking_df=pd.read_csv('../1.데이터/2.usedata/공영주차장데이터.csv',engine='python')
land_df=pd.read_csv('../1.데이터/2.usedata/시유지데이터.csv',engine='python')

### 분석 1
#### 현황분석 진행으로 분석 배경 및 필요성 확보 => 불법주차가 저녁시간(퇴근시간)에 많고 학교 하교시간과 겹치는 특성 도출

1. 시간대별 민원접수 현황(불법주정차 현황)

In [ ]:
tmp=complaints_df.groupby('시',as_index=False)['연도'].count()
tmp.columns=['시','민원수']
tmp['시']=tmp['시'].astype(str)+'시'

In [ ]:
fig = px.treemap(tmp, path=['시'],values='민원수',color='민원수',color_continuous_scale='orrd')
fig.update_traces(textinfo='label+value')

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.show()

2. 직장인/학생 평균 퇴근/하교 시간 & 교통수단

교통수단 통계 - 서울 열린데이터광장

In [ ]:
traffic=pd.read_csv('../1.데이터/2.usedata/교통수단데이터.csv',engine='python')

In [ ]:
traffic=pd.melt(traffic,id_vars='소득별',var_name='교통수단',value_name='이용량')

In [ ]:
traffic['교통수단_중복']=traffic['교통수단'].str.split('+').str[1]
traffic['교통수단']=traffic['교통수단'].str.split('+').str[0]

In [ ]:
tmp=traffic[traffic['교통수단_중복'].isna()==False][['소득별','교통수단_중복','이용량']]
tmp.columns=['소득별','교통수단','이용량']

In [ ]:
traffic=pd.concat([traffic[['소득별','교통수단','이용량']],tmp])

In [ ]:
tmp=traffic.groupby('교통수단',as_index=False)['이용량'].sum()

In [ ]:
tmp['이용량']=round(tmp['이용량']).astype(int)

In [ ]:
fig = px.pie(tmp[tmp['이용량']>10],values='이용량',names='교통수단',width=700,height=700,hole=0.45)

fig.update_traces(textposition='auto', textinfo='percent+label')
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.update_layout(font=dict(size=35))

fig.update_layout(showlegend=False)

fig.show()

In [ ]:
서울시_통근인구=8485850 #서울 열린데이터광장
print('버스 출근인구 : '+str(서울시_통근인구*0.338))
print('도보 출근인구 : '+str(서울시_통근인구*0.186))
print('지하철 출근인구 : '+str(서울시_통근인구*0.154))
print('승용차 출근인구 : '+str(서울시_통근인구*0.127))

1. 초등학교 수업종료시간 - 14:20분<br>
2. 중학교 수업종료시간 - 15:50분<br>
3. 고등학교 수업종료시간 - 16:00분<br>
4. 직장인 평균 퇴근시간 -18:37분

In [ ]:
tmp=pd.DataFrame({'유형':['초등학생','중학생','고등학생','직장인'],'퇴근_하교시간':['14.20','15.50','16.00','18.37']})

In [ ]:
fig = px.bar(tmp, x='유형', y=tmp['퇴근_하교시간'].astype(float),color=tmp['퇴근_하교시간'].astype(float), text=tmp['퇴근_하교시간'].astype(str).str.replace('.',':')+'분',color_continuous_scale='orrd')

fig.update_layout(yaxis_tickformat = ',')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(font=dict(size=20))

fig.show()

서울 데이터만 추출

In [ ]:
시군구=gpd.read_file('../1.데이터/2.usedata/CTPRVN_20220324/ctp_rvn.shp',encoding='euc-kr')

In [ ]:
시군구=시군구[시군구['CTPRVN_CD']=='11']

In [ ]:
시군구=시군구.to_crs("epsg:4326")

In [ ]:
complaints_df=gpd.GeoDataFrame(complaints_df,geometry=gpd.points_from_xy(complaints_df.경도,complaints_df.위도),crs="EPSG:4326")

In [ ]:
complaints_df_seoul = gpd.sjoin(complaints_df, 시군구)

#### 시간적 범위 산출

In [ ]:
tmp=complaints_df_seoul.groupby(['월','일'],as_index=False)['연도'].count()

In [ ]:
tmp['월일']=tmp['월'].astype(str)+'/'+tmp['일'].astype(str)

In [ ]:
tmp[tmp['월일']=='10/7']

In [ ]:
fig.update_yaxes(showline=True,showgrid=True,showticklabels=True, tickfont=dict(size=13) , 
                 gridcolor='#DCEAF2', range=[0, 150] )
fig.update_xaxes(showline=True,showgrid=True,showticklabels=True, tickfont=dict(size=13) )

In [ ]:
fig = px.line(tmp, x='월일', y='연도')

fig.add_trace(go.Scatter(
                    x=['10/7'], 
                    y=[375], 
                    mode='text+markers',  showlegend=False,
                    text = '10월 7일 370건',
                    textposition="top center",marker=dict(size=15, color='Red', symbol=18 )
                    #mode='lines' ,  marker_color='cornflowerblue'
                        )
             ) 
fig.update_layout(yaxis_tickformat = ',')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(font=dict(size=15))
fig.update_yaxes(showline=True,showgrid=True,showticklabels=True, tickfont=dict(size=13) , 
                 gridcolor='#DCEAF2')
fig.update_xaxes(showline=True,showgrid=True,showticklabels=False, tickfont=dict(size=13) )
fig.update_layout({ 'plot_bgcolor': 'rgb(255, 255, 255)','paper_bgcolor': 'rgb(255, 255, 255)'})
fig.show()

열지도 시각화

In [ ]:
complaints_df_seoul=gpd.read_file('../1.데이터/2.usedata/complaints_df_seoul.shp',encoding='euc-kr')

In [ ]:
#공간적 범위 선택
fig = px.density_mapbox(complaints_df_seoul, lat='위도', lon='경도', radius=2,color_continuous_scale='orrd',
                        center=dict(lat=37.55, lon=126.98), zoom=10.65,opacity=0.2, 
#                         mapbox_style="stamen-terrain"
#                        mapbox_style="open-street-map"
                       mapbox_style="carto-positron"
                        )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_coloraxes(showscale=False)
# po.write_html(fig,file='반지름1히트맵.html')

fig.show()

### 분석 3
#### 분석5 전체 분석 전 최단거리 알고리즘 활용 기법 검증을 위해 필요

한두개 학교만 출력

In [ ]:
G = ox.load_graphml('../1.데이터/2.usedata/서울경기도_도로.graphml')

In [ ]:
#출발지 및 도착지 좌표
def get_node(y,x):
    return ox.get_nearest_node(G, (y, x))

In [ ]:
def get_length(orig_node,dest_node):
    len_node = nx.shortest_path_length(G, orig_node, dest_node, weight='length') / 1000
    return len_node

In [ ]:
# 학교데이터 범위 1차산정
def get_near_school(범위지수,y,x):
    tmp_school_df=school_df[(school_df['X좌표']<x+범위지수)&(school_df['X좌표']>x-범위지수)&(school_df['Y좌표']<y+범위지수)&(school_df['Y좌표']>y-범위지수)]
    if len(tmp_school_df)!=0:
        return tmp_school_df
    
    범위지수+=0.001
    return get_near_school(범위지수,y,x)

### 예시 서울시청

In [ ]:
학교=pd.DataFrame({'경도':[126.9808065],'위도':[37.5661367]})

서대문구청, 마포구청, 영등포구청, 관악구청, 서초구청, 송파구청, 중량구청, 강북구청, 종로구청

In [ ]:
예시점=pd.DataFrame({'경도':[126.9368157,126.9019551,126.8962717,126.9515619,127.0327323,127.1059177,127.0931523,127.0255381,126.97899],'위도':[37.57916186,37.56621419,37.52636177,37.47836838,37.48358701,37.51458372,37.6063046,37.63975133,37.57350424]})

In [ ]:
x2_li=[]
y2_li=[]
x_li=[]
y_li=[]
orig_node_li=[]
dest_node_li=[]
length_li=[]

cnt=1
for y2,x2 in zip(예시점['위도'],예시점['경도']):
    print(cnt)
    cnt+=1
        
    x2_li.append(x2)
    y2_li.append(y2)
    orig_node=get_node(y2,x2)
    orig_node_li.append(orig_node)


    dest_node=get_node(학교.위도[0], 학교.경도[0])
    len_node=get_length(orig_node,dest_node)

    x_li.append(학교.경도[0])
    y_li.append(학교.위도[0])
    dest_node_li.append(dest_node)

In [ ]:
장훈고=pd.DataFrame({"출발지_x":x2_li,"출발지_y":y2_li,"도착지_x":x_li,"도착지_y":y_li,"출발지_node":orig_node_li,"도착지_node":dest_node_li})

In [ ]:
long_li=[]
lat_li=[]
for start,destiny in zip(장훈고['출발지_node'],장훈고['도착지_node']):
    route=nx.shortest_path(G, start, destiny, weight='length')
    long = [] 
    lat = []  
    for i in route:
        point = G.nodes[i]
        long.append(point['x'])
        lat.append(point['y'])
    long_li.append(long)
    lat_li.append(lat)
    
장훈고['경로_x']=long_li
장훈고['경로_y']=lat_li

In [ ]:
장훈고.reset_index(drop=True,inplace=True)
장훈고['경로_x']=장훈고['경로_x'].astype(str)
장훈고['경로_y']=장훈고['경로_y'].astype(str)

In [ ]:
# 경로 스케처
fig = go.Figure() 

for i in range(len(장훈고)):

    fig.add_trace(go.Scattermapbox(
        name = "경로",
        mode = "lines",
        lon = json.loads(장훈고['경로_x'][i]),
        lat = json.loads(장훈고['경로_y'][i]),
        marker = {'size': 10},
        line = dict(width = 1, color = 'blue')))

    # 시작지점 스케처
    fig.add_trace(go.Scattermapbox(
        name = "시작지점",
        mode = "markers",
        lon = [장훈고['출발지_x'][i]],
        lat = [장훈고['출발지_y'][i]],
        marker = {'size': 7, 'color':"red"}))

    # 도착지점 스케쳐
    fig.add_trace(go.Scattermapbox(
        name = "도착지점",
        mode = "markers",
        lon = [장훈고['도착지_x'][i]],
        lat = [장훈고['도착지_y'][i]],
        marker = {'size': 15, 'color':'green'}))

#     # 중심에 거리 및 시간 표시
#     lat_center = np.mean(lat)
#     long_center = np.mean(long)

#     fig.add_trace(go.Scattermapbox(
#         name = "거리",
#         text = str(round(len_node,1))+'km',
#         mode = "markers+text",
#         marker = {'size': 12, 'color':'black'},
#         textfont=dict(size=12, color='white'),
#         lon = [long_center],
#         lat = [lat_center]))

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(showlegend=False)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                  mapbox = {
                      'center': {'lat': 37.552, 
                      'lon': 126.995},
                      'zoom': 10.5})
fig.show()

### 서울 전체 학교

In [ ]:
complaints_df_seoul=complaints_df_seoul.sort_values(['경도','위도'])

In [ ]:
x2_li=[]
y2_li=[]
x_li=[]
y_li=[]
orig_node_li=[]
dest_node_li=[]
length_li=[]

cnt=1
for y2,x2 in zip(complaints_df_seoul['위도'],complaints_df_seoul['경도']):
    print(cnt)
    cnt+=1
        
    x2_li.append(x2)
    y2_li.append(y2)
    orig_node=get_node(y2,x2)
    orig_node_li.append(orig_node)
    school_df_tmp=get_near_school(0.00005,y2,x2)

    tmp_var=50000
    for y,x in zip(school_df_tmp.Y좌표,school_df_tmp.X좌표):
        dest_node=get_node(y, x)
        len_node=get_length(orig_node,dest_node)

        if tmp_var<len_node:
            pass
        else:
            tmp_var=len_node
    x_li.append(x)
    y_li.append(y)
    dest_node_li.append(dest_node)
    length_li.append(tmp_var)

In [ ]:
complaints_parking_vf=pd.DataFrame({"출발지_x":x2_li,"출발지_y":y2_li,"도착지_x":x_li,"도착지_y":y_li,"출발지_node":orig_node_li,"도착지_node":dest_node_li,"거리":length_li})

In [ ]:
long_li=[]
lat_li=[]
for start,destiny in zip(complaints_parking_vf['출발지_node'],complaints_parking_vf['도착지_node']):
    route=nx.shortest_path(G, start, destiny, weight='length')
    long = [] 
    lat = []  
    for i in route:
        point = G.nodes[i]
        long.append(point['x'])
        lat.append(point['y'])
    long_li.append(long)
    lat_li.append(lat)
    
complaints_parking_vf['경로_x']=long_li
complaints_parking_vf['경로_y']=lat_li

In [ ]:
complaints_parking_vf.to_csv('../1.데이터/2.usedata/전체_vf.csv',encoding='euc-kr',index=False)

####  시각화

In [ ]:
complaints_parking_vf=pd.read_csv('../1.데이터/2.usedata/전체_vf.csv',engine='python')

In [ ]:
관악구=gpd.read_file('../1.데이터/2.usedata/SIG_20220324/sig.shp',encoding='euc-kr')

In [ ]:
관악구=관악구[관악구['SIG_KOR_NM']=='양천구']

In [ ]:
관악구=관악구.to_crs("epsg:4326")

In [ ]:
complaints_parking_vf=gpd.GeoDataFrame(complaints_parking_vf,geometry=gpd.points_from_xy(complaints_parking_vf.도착지_x,complaints_parking_vf.도착지_y),crs="EPSG:4326")

In [ ]:
complaints_df_관악 = gpd.sjoin(complaints_parking_vf, 관악구)

In [ ]:
complaints_df_관악.reset_index(drop=True,inplace=True)

In [ ]:
def color_change(i):
    cor_li=['aliceblue','antiquewhite','aqua','aquamarine','azure','beige','bisque','black','blanchedalmond','blue','blueviolet','brown','burlywood','cadetblue','chartreuse','chocolate','coral','cornflowerblue','cornsilk','crimson','cyan','darkblue','darkcyan','darkgoldenrod','darkgray','darkgrey','darkgreen','darkkhaki','darkmagenta','darkolivegreen','darkorange','darkorchid','darkred','darksalmon','darkseagreen','darkslateblue','darkslategray','darkslategrey','darkturquoise','darkviolet','deeppink','deepskyblue','dimgray','dimgrey','dodgerblue','firebrick','floralwhite','forestgreen','fuchsia','gainsboro','ghostwhite','gold','goldenrod','gray','grey','green','greenyellow','honeydew','hotpink','indianred','indigo','ivory','khaki','lavender','lavenderblush','lawngreen','lemonchiffon','lightblue','lightcoral','lightcyan','lightgoldenrodyellow','lightgray','lightgrey','lightgreen','lightpink','lightsalmon','lightseagreen','lightskyblue','lightslategray','lightslategrey','lightsteelblue','lightyellow','lime','limegreen','linen','magenta','maroon','mediumaquamarine','mediumblue','mediumorchid','mediumpurple','mediumseagreen','mediumslateblue','mediumspringgreen','mediumturquoise','mediumvioletred','midnightblue','mintcream','mistyrose','moccasin','navajowhite','navy','oldlace','olive','olivedrab','orange','orangered','orchid','palegoldenrod','palegreen','paleturquoise','palevioletred','papayawhip','peachpuff','peru','pink','plum','powderblue','purple','red','rosybrown','royalblue','rebeccapurple','saddlebrown','salmon','sandybrown','seagreen','seashell','sienna','silver','skyblue','slateblue','slategray','slategrey','snow','springgreen','steelblue','tan','teal','thistle','tomato','turquoise','violet','wheat','white','whitesmoke','yellow','yellowgreen']
    return cor_li[i]

In [ ]:
# 경로 스케처
fig = go.Figure() 

for i in range(len(complaints_df_관악)):

    fig.add_trace(go.Scattermapbox(
        name = "경로",
        mode = "lines",
        lon = json.loads(complaints_df_관악['경로_x'][i]),
        lat = json.loads(complaints_df_관악['경로_y'][i]),
        marker = {'size': 10},
        line = dict(width = 1, color = 'blue')))

    # 시작지점 스케처
    fig.add_trace(go.Scattermapbox(
        name = "시작지점",
        mode = "markers",
        lon = [complaints_df_관악['출발지_x'][i]],
        lat = [complaints_df_관악['출발지_y'][i]],
        marker = {'size': 7, 'color':"red"}))

    # 도착지점 스케쳐
    fig.add_trace(go.Scattermapbox(
        name = "도착지점",
        mode = "markers",
        lon = [complaints_df_관악['도착지_x'][i]],
        lat = [complaints_df_관악['도착지_y'][i]],
        marker = {'size': 7, 'color':'green'}))

#     # 중심에 거리 및 시간 표시
#     lat_center = np.mean(lat)
#     long_center = np.mean(long)

#     fig.add_trace(go.Scattermapbox(
#         name = "거리",
#         text = str(round(len_node,1))+'km',
#         mode = "markers+text",
#         marker = {'size': 12, 'color':'black'},
#         textfont=dict(size=12, color='white'),
#         lon = [long_center],
#         lat = [lat_center]))

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(showlegend=False)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                  mapbox = {
                      'center': {'lat': 37.475, 
                      'lon': 126.94},
                      'zoom': 13})
fig.show()

### 분석 
#### 최종 분석 결과 도출

중심점 = 학교 반경을 유동적으로 준다음 주차면수에 비해 주정차 신고값이 많을경우 그 점들에대해 최단거리 구해서 POP INSERT 하는방식

In [ ]:
# 경로 스케처
fig = go.Figure() 

for i in range(len(complaints_parking_vf)):

    fig.add_trace(go.Scattermapbox(
        name = "경로",
        mode = "lines",
        lon = json.loads(complaints_parking_vf['경로_x'][i]),
        lat = json.loads(complaints_parking_vf['경로_y'][i]),
        marker = {'size': 10},
        line = dict(width = 1, color = 'blue')))

    # 시작지점 스케처
    fig.add_trace(go.Scattermapbox(
        name = "시작지점",
        mode = "markers",
        lon = [complaints_parking_vf['출발지_x'][i]],
        lat = [complaints_parking_vf['출발지_y'][i]],
        marker = {'size': 4, 'color':"red"}))

    # 도착지점 스케쳐
    fig.add_trace(go.Scattermapbox(
        name = "도착지점",
        mode = "markers",
        lon = [complaints_parking_vf['도착지_x'][i]],
        lat = [complaints_parking_vf['도착지_y'][i]],
        marker = {'size': 4, 'color':'green'}))

#     # 중심에 거리 및 시간 표시
#     lat_center = np.mean(lat)
#     long_center = np.mean(long)

#     fig.add_trace(go.Scattermapbox(
#         name = "거리",
#         text = str(round(len_node,1))+'km',
#         mode = "markers+text",
#         marker = {'size': 12, 'color':'black'},
#         textfont=dict(size=12, color='white'),
#         lon = [long_center],
#         lat = [lat_center]))

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(showlegend=False)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                  mapbox = {
                      'center': {'lat': 37.55, 
                      'lon': 126.98},
                      'zoom': 10.65})
fig.show()

In [ ]:
complaints_parking_vf['평균거리']=complaints_parking_vf['거리'].copy()

In [ ]:
tmp=complaints_parking_vf.groupby(['도착지_x','도착지_y'],as_index=False).agg({'출발지_x':'count','평균거리':'mean'})

In [ ]:
tmp=tmp.sort_values('출발지_x',ascending=False)

In [ ]:
school_df.loc[school_df['학교명'].str.contains('영풍초'),'X좌표']=127.14315516161001
school_df.loc[school_df['학교명'].str.contains('묘곡초'),'Y좌표']=37.558554728522104
school_df.loc[school_df['학교명'].str.contains('화계초'),'X좌표']=127.02906701433999
school_df.loc[school_df['학교명'].str.contains('봉은초'),'X좌표']=127.06087214626001
school_df.loc[school_df['학교명'].str.contains('옥정초'),'Y좌표']=37.542009825996104
school_df.loc[school_df['학교명'].str.contains('강남초'),'X좌표']=126.95273815978999

In [ ]:
tmp=pd.merge(tmp,school_df[['학교명','X좌표','Y좌표','주차가능대수']],how='left',left_on=['도착지_x','도착지_y'],right_on=['X좌표','Y좌표'])

In [ ]:
tmp=tmp[['학교명','X좌표','Y좌표','주차가능대수','출발지_x','평균거리']]
tmp.columns=['학교명','X좌표','Y좌표','주차가능대수','민원수','평균거리']

In [ ]:
tmp['불법주정차 유입수']=tmp['민원수']*1

In [ ]:
tmp['여유주차가능대수']=tmp['주차가능대수']-tmp['불법주정차 유입수']

In [ ]:
tmp['평균시간']=tmp['평균거리']/4.8*60

In [ ]:
tmp=tmp.sort_values('불법주정차 유입수',ascending=False).drop(['X좌표','Y좌표','민원수'],axis=1)

In [ ]:
tmp=tmp[['학교명','주차가능대수','불법주정차 유입수','여유주차가능대수','평균거리','평균시간']]

In [ ]:
tmp.columns=['학교명', '주차가능대수', '불법주정차 유입수', '여유주차가능대수', '평균거리(km)', '평균시간(분)']

In [ ]:
tmp['평균거리(km)']=round(tmp['평균거리(km)'],2)
tmp['평균시간(분)']=round(tmp['평균시간(분)'],1)

In [ ]:
tmp.head(10)['평균시간(분)'].mean()

In [ ]:
len(set(school_df['학교명']))

In [ ]:
#민원통계=pd.DataFrame({'연도':[2018,2019,2020,2021],  '민원수':[747362,970166,1030074,1107648]    }) #경찰청
민원통계=pd.DataFrame({'연도':[2016,2017,2018,2019,2020,2021],  '민원수':[128988,500739,475072,1088333,1755887,3437246]    }) #서울스마트신고
#민원통계=pd.DataFrame({'연도':[2016,2017,2018,2019,2020,2021],  '민원수':[1,3,5,7,9,100]    }) #서울스마트신고

민원통계.set_index('연도',inplace=True)


민원통계 = 민원통계/10000
민원통계 = sm.add_constant(민원통계[['민원수']])


sub = 민원통계[['const']]
sub['연도'] = sub.index
sub=sub[['const','연도']]


model = sm.OLS(민원통계[['민원수']],sub)
result = model.fit()



pred = pd.DataFrame({'const':[1.0]*(10),'연도':range(2016,2026)} )

pred['민원수'] = result.predict(pred)
pred = pred.drop('const', axis=True).set_index('연도')

In [ ]:
### 시각화

fig = go.Figure()
# 사업 전 막대그래프
fig.add_trace(go.Bar(
                     x=민원통계[(민원통계.index < 2022)  ].index.astype(str), 
                     y=민원통계[(민원통계.index < 2022)  ].민원수,
                     text = round(민원통계.민원수),
                     name = '민원수' , marker_color='#3A9EF0',
                     textposition="auto"
                    )
             ) 

# 사업 전 예측  ( 점선 )
fig.add_trace(go.Scatter(
                    x=pred.index.astype(str), 
                    y=pred.민원수, 
                    name ='민원수예측', 
                    line=dict(color='#1B57C8', width=4, dash='dash')
                    #mode='lines' ,  marker_color='cornflowerblue'
                        )
             ) 


# 사업 중 예측  ( 마커 )
fig.add_trace(go.Scatter(
                    x=pred[pred.index == 2025 ].index.astype(str), 
                    y=pred[pred.index == 2025 ].민원수, 
                    mode='text+markers',name='예측값(민원수)',   showlegend=False,
                    text = round(pred[pred.index == 2025 ].민원수),
                    textposition="top center",marker=dict(size=20, color='#1B57C8', symbol=18 )
                    #mode='lines' ,  marker_color='cornflowerblue'
                        )
             ) 




fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0},
                    font=dict(family="Malgun Gothic",size=13,color='#000000'),   
                    title=dict( text='<b> 회귀선 </b>',  x=0.5 ),
                    legend=dict(orientation="h",yanchor="bottom", x=0.5, y=-0.15, xanchor="center")
                         )
fig.update_yaxes(showline=True,showgrid=True,showticklabels=True, tickfont=dict(size=13) , 
                 gridcolor='#DCEAF2' 
                )
fig.update_xaxes(showline=True,showgrid=True,showticklabels=True, tickfont=dict(size=13) )
fig.update_layout({ 'plot_bgcolor': 'rgb(255, 255, 255)','paper_bgcolor': 'rgb(255, 255, 255)'})
fig.show()


#arima_시각화(pred_df,target)